In [1]:
import duckdb
import pandas as pd
import numpy as np
from src.model.utils import train_model
from src.preprocess.etl import get_dataframe
from src.model.inference import predict_month
from src.constants import PATH_DATABASE, MONTHS_BASELINE, MONTHS_INFERENCE, PARAMS, PATH_FINAL


import warnings

warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
con = duckdb.connect(database=PATH_DATABASE, read_only=True)

In [4]:
all_months = MONTHS_BASELINE + MONTHS_INFERENCE
all_months = [str(month) for month in all_months]
where_clause = ", ".join(all_months)
where_clause

'202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109'

In [5]:
df = con.sql(
    f"""
    SELECT
        *
    FROM read_parquet('{PATH_FINAL}')
    WHERE foto_mes IN ({where_clause})
    """
).to_df()

In [6]:
df.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria_202104,clase_ternaria_202105,clase_ternaria_202106,clase_ternaria_202107,clase_ternaria_202108,clase_ternaria_202109
0,86787796,202109,1,0,0,62,170,398.33,13229.56,453.73,...,9944.15,5.0,0.0,950.13,None,None,None,None,None,BAJA+1
1,86788155,202109,1,0,0,40,170,1889.50,10379.18,2001.33,...,5177.03,3.0,0.0,598.23,None,None,None,None,None,BAJA+1
2,86790136,202109,1,0,0,61,170,965.88,7587.93,1396.01,...,11131.17,10.0,0.0,1853.34,None,None,None,None,None,BAJA+1
3,86791400,202109,1,0,0,35,170,5114.99,82662.36,1044.59,...,26033.78,8.0,0.0,4316.64,None,None,None,None,None,BAJA+1
4,86791878,202109,1,0,0,42,170,-6128.26,-125656.61,3352.77,...,81208.53,4.0,0.0,18709.35,None,None,None,None,None,BAJA+1


In [7]:
np.sort(df["foto_mes"].unique())

array([202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102,
       202103, 202104, 202105, 202106, 202107, 202108, 202109])

In [8]:
drop_cols = [f"clase_ternaria_{i}" for i in MONTHS_INFERENCE]
drop_cols.append("clase_binaria")
drop_cols

['clase_ternaria_202104',
 'clase_ternaria_202105',
 'clase_ternaria_202106',
 'clase_ternaria_202107',
 'clase_ternaria_202108',
 'clase_ternaria_202109',
 'clase_binaria']

In [9]:
ground_truth = df[["numero_de_cliente","foto_mes", "clase_ternaria_202109"]]
ground_truth = ground_truth[ground_truth["foto_mes"]]
ground_truth

,numero_de_cliente,foto_mes,clase_ternaria_202109
46435,73139652,202105,CONTINUA
46436,73569089,202105,CONTINUA
46437,73785768,202105,CONTINUA
46438,73804974,202105,CONTINUA
46439,73918262,202105,CONTINUA
...,...,...,...
2425602,83503713,202104,CONTINUA
2425603,83505182,202104,CONTINUA
2425604,83505370,202104,CONTINUA
2425605,83506856,202104,CONTINUA


In [10]:
ground_truth["ganancia"] = ground_truth["clase_ternaria_202109"].map({"BAJA+2":270000, "BAJA+1":-7000, "CONTINUA":-7000})
ground_truth.head()

,numero_de_cliente,foto_mes,clase_ternaria_202109,ganancia
46435,73139652,202105,CONTINUA,-7000
46436,73569089,202105,CONTINUA,-7000
46437,73785768,202105,CONTINUA,-7000
46438,73804974,202105,CONTINUA,-7000
46439,73918262,202105,CONTINUA,-7000


In [11]:
for month in MONTHS_INFERENCE:
    print("Pred month:", month)
    tags_monthly = {
        'stage': 'monthly',
        'last_month': month,
    }

    
    df["clase_binaria"] = df[f"clase_ternaria_{month}"].map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})
    
    df_test = df[df["foto_mes"].isin([month])].copy()
    df_train = df[df["foto_mes"] <= int(month - 2)].copy()

    print("Training months:", df_train["foto_mes"].unique())
    
    df_test = df_test.reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    
    X_train = df_train.drop(columns=drop_cols, axis=1).copy()
    X_test = df_test.drop(columns=drop_cols, axis=1).copy()
    
    y_train_binaria = df_train["clase_binaria"].copy()
    
    y_real_ternaria = df_test["clase_ternaria_202109"].copy().map({"BAJA+2":1, "BAJA+1":0, "CONTINUA":0})
    y_real_binaria = df_test["clase_ternaria_202109"].copy().map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})

    if month < 202108:
        model = train_model(X_train, y_train_binaria, X_test, y_real_ternaria, y_real_binaria, "monthly", PARAMS, tags_monthly)
    else:
        model = train_model(X_train, y_train_binaria, None, None, None, "monthly", PARAMS, tags_monthly)

    preds = X_test[["numero_de_cliente", "foto_mes"]].copy()
    preds["prediction"] = predict_month(model, X_test)
    preds = preds.merge(ground_truth, on=["numero_de_cliente", "foto_mes"])

    preds = preds.sort_values(["prediction"], ascending=False)
    
    gan_mes = preds["prediction"] * preds["ganancia"]
    print("Ganancia", month, gan_mes.sum())
    
    preds.to_csv(f"~/buckets/b1/exp_colab/datasets/processed/predictions/monthly/preds_{month}.csv", index=False)

Pred month: 202104
Training months: [202010 202007 202011 202012 202102 202009 202008 202101]


2023/11/16 03:01:27 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 93db84b771fb4760b737dfdd7d8dcc01. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:01:31 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:01:31 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202104 -26682829.987446956
Pred month: 202105
Training months: [202010 202103 202007 202011 202012 202102 202009 202008 202101]


2023/11/16 03:04:04 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 399017c1f39443e38f8a1d19b71cc379. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:04:07 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:04:07 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202105 -68116833.9529853
Pred month: 202106
Training months: [202104 202010 202103 202007 202011 202012 202102 202009 202008 202101]


...

2023/11/16 03:13:47 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 382b5bb9d5cf419782f3b3aaec554096. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:13:49 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:13:49 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202108 0.0
Pred month: 202109
Training months: [202105 202104 202010 202103 202007 202011 202012 202106 202102 202107
 202009 202008 202101]


2023/11/16 03:17:13 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID fcefdafc1dbb466f90953e43e9c5e86f. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:17:16 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:17:16 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202109 0.0


<Figure size 1050x700 with 0 Axes>

## Sin `foto_mes`

In [12]:
if True:
    drop_cols.append("foto_mes")

In [13]:
for month in MONTHS_INFERENCE:
    print("Pred month:", month)
    tags_monthly = {
        'stage': 'monthly_sin_foto_mes',
        'last_month': month,
    }

    
    df["clase_binaria"] = df[f"clase_ternaria_{month}"].map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})
    
    df_test = df[df["foto_mes"].isin([month])].copy()
    df_train = df[df["foto_mes"] <= int(month - 2)].copy()

    print("Training months:", df_train["foto_mes"].unique())
    
    df_test = df_test.reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    
    X_train = df_train.drop(columns=drop_cols, axis=1).copy()
    X_test = df_test.drop(columns=drop_cols, axis=1).copy()
    
    y_train_binaria = df_train["clase_binaria"].copy()
    
    y_real_ternaria = df_test["clase_ternaria_202109"].copy().map({"BAJA+2":1, "BAJA+1":0, "CONTINUA":0})
    y_real_binaria = df_test["clase_ternaria_202109"].copy().map({"BAJA+2":1, "BAJA+1":1, "CONTINUA":0})

    if month < 202108:
        model = train_model(X_train, y_train_binaria, X_test, y_real_ternaria, y_real_binaria, "monthly", PARAMS, tags_monthly)
    else:
        model = train_model(X_train, y_train_binaria, None, None, None, "monthly", PARAMS, tags_monthly)

    preds = X_test.copy()
    preds["foto_mes"] = month
    preds = preds[["numero_de_cliente", "foto_mes"]]
    preds["prediction"] = predict_month(model, X_test)
    preds = preds.merge(ground_truth, on=["numero_de_cliente", "foto_mes"])
    
    preds = preds.sort_values(["prediction"], ascending=False)
    
    gan_mes = preds["prediction"] * preds["ganancia"]
    print("Ganancia", month, gan_mes.sum())
    
    preds.to_csv(f"~/buckets/b1/exp_colab/datasets/processed/predictions/monthly/preds_{month}_sin_foto_mes.csv", index=False)

Pred month: 202104
Training months: [202010 202007 202011 202012 202102 202009 202008 202101]


2023/11/16 03:19:02 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 89596203ff7f4a0f8eea47007c367e15. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:19:04 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:19:04 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202104 3839658.91075169
Pred month: 202105
Training months: [202010 202103 202007 202011 202012 202102 202009 202008 202101]


2023/11/16 03:21:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 1448e036d9d7432da20d133306ace540. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:21:37 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:21:37 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202105 -53408418.306852356
Pred month: 202106
Training months: [202104 202010 202103 202007 202011 202012 202102 202009 202008 202101]


...

2023/11/16 03:31:30 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 57b99db202b74eb786bd97364ffe3675. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:31:32 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:31:32 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202108 0.0
Pred month: 202109
Training months: [202105 202104 202010 202103 202007 202011 202012 202106 202102 202107
 202009 202008 202101]


2023/11/16 03:34:59 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 9aef653dfa4c46c899c455685bde4517. Failed operations: [MlflowException("Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
2023/11/16 03:35:01 WARNING mlflow.utils.requirements_utils: Found datatable version (1.1.0a0+build.1695691407.jauriberamirez) contains a local version label (+build.1695691407.jauriberamirez). MLflow logged a pip requirement for this package as 'datatable==1.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/16 03:35:01 WARNING mlflow.utils.requirements_utils: Found

Ganancia 202109 0.0


<Figure size 1050x700 with 0 Axes>